# Lab 3


In [60]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor
import numpy as np

In [61]:
# variables
OFFSPRING_SIZE = 20
K= 5
POPULATION_SIZE = 10
TOURNAMENT_SIZE = 5

# The Nim and Nimply classes

In [62]:
Nimply = namedtuple("Nimply", "row, num_objects") # move

In [63]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

# Rule based agent using Nim-Sum


In [64]:
def nim_sum_l(nim):
  X = nim._rows[0]
  # X is the nim_sum(bitwise xor) of all heap sizes
  for i in range(1, len(nim._rows)):
    X = X ^ nim._rows[i]

  nim_sum_val = []
  # calculate the nim_sum between X and each heap size
  for i in nim._rows:
    val = i ^ X
    nim_sum_val.append(val)

  row = "false"
 
  for i in range(len(nim_sum_val)):
    if nim_sum_val[i] < nim._rows[i]:
      row = i
      break
  
  # reduce that heap to value nim_sum
  if (row != "false"):
    num_objects = nim._rows[row] - nim_sum_val[row]
    move = Nimply(row, num_objects)
  
  else:
    rand_row = random.randrange(0,len(nim._rows))

    while(nim._rows[rand_row] == 0):
      rand_row = random.randrange(0,len(nim._rows))

    if(nim._rows[rand_row] != 1):
      rand_obj = random.randrange(1, nim._rows[rand_row])
                                  
    else: rand_obj = 1
    move = Nimply(rand_row, rand_obj)
  return move

# Evolved Strategy

## Parameters
- How many rows that are still alive <br />
- How many of the inital sticks are left <br />
- <br />


## Rule
- Defence: If stick < 5% - take only 1 stick <br />
-  <br />

     0 the middle column
     pick one from the longest
     pick one from random rows
     zero the smallest row
     half the biggest row
     take k sticks if possible otherwise take the max <br />

     onerow-stategy: only one row - leave x
     2 rows: 1 rad = 1 element. ta x element från den andra raden
     2 rows: both has more than 1 element - take random
     else: take random random. 

     genome: dict 
     individ = {rule1: x, rule2:y, rule3:z, rule4:n}

In [65]:
MATCHES = 500


def strategy():
    if one_row:
        return one_row_start()

In [66]:
#Remove x element from the only row left
# 0 < x <= k
def one_row_strategy(state: Nim, individual) -> Nimply:
    row_counter = -1
    for row in state.rows: 
        row_counter += 1
        if row > 0:
            return Nimply(row_counter , min(row, individual[0]))


In [67]:
def two_rows_strategy1(nim, individual):
    return None

In [68]:
def count_rows_alive(nim):
    rows_alive = 0
    for i in nim._rows:
        if nim._rows == 0:
            count += 1
    return rows_alive

In [69]:
def zero_middle_column(nim):
    #find the row with the amount of sticks that are in the middle
    sorted_rows = sorted(nim._rows)

    middle_row = int(len(sorted_rows)/2) #make sure it is an even number

    obj = sorted_rows[middle_row]

    for i in nim._rows:
        if nim._rows[i] == obj:
            final_row = i

    move = Nimply(i, obj)
    
    return move


# MinMax


In [70]:
def all_possible_moves(nim):
    possible_moves = []

    for row in range(len(nim._rows)):
        for j in range(nim._rows[row]):
            copy_row = deepcopy(row)

            new_move = copy_row[row] - j - 1
            possible_moves.append(new_move)

    return possible_moves
            

In [71]:
def evaluate(nim, player):
    if not nim: # if game is finished
        # player 0 loose if return -1
        return -1 if player == 0 else 1


# Sample strategies

In [72]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [73]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result

In [74]:
def active_rows_index(state: Nim) -> list:
    active_rows_index = []
    count = 0
    for o in state.rows:
        if o > 0:
          active_rows_index.append(count)
        count += 1
    return active_rows_index

In [75]:
def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["active_rows_index"] = active_rows_index(state)
    cooked["rows_with_one_element"] = [(index, r) for index, r in enumerate(state.rows) if r == 1]
    
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)
    
    #cooked["one_row_left"] = one_row_strategy(state)
    cooked["pure_random"] = pure_random(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

In [76]:
def rule_two(state: Nim, data:dict, genome: dict):
  active_rows_index = data["active_rows_index"]
  single_elem_rows = data["rows_with_one_element"]

  if genome["Rule2"][0] == 0:  # take from row with one elem
    elem = 1 # want to take the last elem in row
    single_elem_row = single_elem_rows[0][0] # looks like [(row,elem),(row,elem)] if two rows with single elem
    ply = Nimply(single_elem_row, 1)

  else: #take from the row with more than one element
    if len(single_elem_rows) > 1:
      row = single_elem_rows[0][0]
    else:
      row = [i for i in active_rows_index if state.rows[i] != 1][0]
    
    if (genome['Rule2'][1] > row): # if it wants to leave more elem than exists in row
      elem = 1
      ply = Nimply(row, 1)
                      
    else: 
      elem = max(state.rows[row] - genome['Rule2'][1], 1)
      ply = Nimply(row, elem)
      
  return ply

In [77]:
def rule_three(state: Nim, data: dict, genome: dict):
  #logging.debug('Coosing rule 3')
  if (genome['Rule3'][0] == 0): # choose from row with fewest elemt
    row = data['shortest_row']
  else:
    row = data['longest_row']
      
  elem = max(state.rows[row] - genome['Rule3'][1], 1)
  ply = Nimply(row, elem)

  return ply

In [78]:
def two_rows_left(state: Nim, data: dict, genome: dict ) -> Nimply:
  
  rows_with_one_elem = data["rows_with_one_element"]
 
  if rows_with_one_elem: # Rule 2
    #logging.debug("Choosing rule 2")
    ply = rule_two(state=state, data=data, genome=genome)
  
  else: # Rule 3              
    ply = rule_three(state, data, genome)

  return ply

In [97]:
def one_row_left(state: Nim, data: dict, genome: dict) -> Nimply:
    
    #Rule 1 - if only one row left: leave x sticks
    active_row = data["active_rows_index"][0]
    elem_last_row = state.rows[active_row] #active_rows_index returns a list -> need to get the first one
   
    if elem_last_row < genome['Rule1']: #if the rule want to leave more sticks than exists in row
        ply = Nimply(active_row, 1)
        
    else:
        #add if you want to take more than K - take k
        #if (k <)
        
        elem_to_remove = max(elem_last_row - genome["Rule1"], 1) #if take more elem than exists -> take all
        ply = Nimply(active_row, elem_to_remove)

    return ply
  

In [80]:
def make_strategy(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        active_rows_number = data["active_rows_number"]

        #only one active row left
        if active_rows_number == 1:
            #logging.debug('Choosing rule 1')
            ply = one_row_left(state=state,data=data, genome=genome) 
        
        elif active_rows_number == 2:
           ply = two_rows_left(state=state, data=data, genome=genome)
      
        else:
            move = random.choice(data["possible_moves"])
            ply = Nimply(move[0], move[1])

        return ply

    return evolvable

In [99]:
nim = Nim(3)

ply = Nimply(2,5)
nim.nimming(ply)
ply = Nimply(1,3)
nim.nimming(ply)

data = cook_status(nim)

print(nim)

genome = {'Rule1': 1, 'Rule2': [0,3], 'Rule3': [1,10]}

pl = make_strategy(genome=genome)
ply = pl(nim)
print(ply)
nim.nimming(ply)

#print("rows_with_one_elem", data["rows_with_one_element"])

#print("rdfjh",state.rows[data["active_rows_index"][0]])

#one_match = [(index, r) for index, r in enumerate(state.rows) if r == 1]

DEBUG:root:Choosing rule 1


<1 0 0>
Nimply(row=0, num_objects=1)


In [82]:
from tqdm import tqdm
#compute the fittness
def head2head(individual1: dict, individual2: dict, nim_size: int):
    won = 0
    NUM_MATCHES = 1 
    nim = Nim(nim_size)

    players = (make_strategy(individual1), make_strategy(individual2))
    player = 0
    
    print("H2H function")
    print(f"status: Initial board  -> {nim}")
    while nim:
        ply = players[player](nim)
        nim.nimming(ply)
        print(f"status: After player {player} -> {nim}")
        player = 1 - player
    winner = 1 - player
    if winner == 0:
        print("winner ind 1")
        individual1["fitness"] +=1
        #öka fitness på individ1 vid vinst
    else: 
        individual2["fitness"] +=1
        print("winner ind 2")
        #öka fitness på individ2 vid vinst
    
   


You should consider upgrading via the 'C:\Users\angel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [101]:
indv1 = {'Rule1': 2, 'Rule2': [1,3], 'Rule3': [1,10], 'fitness':0}
indv2 = {'Rule1': 0, 'Rule2': [1,2], 'Rule3': [2,2], 'fitness':0}

head2head(indv1, indv2, 3)




DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1


H2H function
status: Initial board  -> <1 3 5>
status: After player 0 -> <1 3 4>
status: After player 1 -> <1 3 2>
status: After player 0 -> <1 1 2>
status: After player 1 -> <1 1 0>
status: After player 0 -> <0 1 0>
status: After player 1 -> <0 0 0>
winner ind 2


In [84]:
# calculate fitness for whole population
def calculate_fitness(state: Nim, population: list, nim_size):
    for p1 in population:
        for p2 in population:
            head2head(individual1=p1, individual2=p2, nim_size=nim_size)


In [85]:

def init_population(nim_size: int) -> list:
    population = []
    max_leave = (nim_size-1)*2 # last row of the table will have nim_size*2-1 objects

    cond = POPULATION_SIZE
    
    while cond:
        individual = {'Rule1': random.randint(0,max_leave), 'Rule2': [random.randint(0,1), max_leave], 'Rule3': [random.randint(0, 1), random.randint(0, max_leave)]}
        individual['fitness'] = 0
        population.append(individual)
        cond -= 1
    return population

In [86]:
def k_fittest_indv(population: list, k: int) -> list:
    return sorted(population, key=lambda indv: indv['fitness'], reverse=True)[:k]


In [87]:
def clean_fitness(individuals: list):
    for indv in individuals:
        indv['fitness'] = 0

In [88]:
# select k random individuals, return the one with best fitness
def tournament(population: list, nim_size: int) -> dict:    
    contestors = random.sample(population, TOURNAMENT_SIZE)
    clean_fitness(contestors)
    print("H2H startes")
    for p1 in contestors:
        for p2 in contestors:
            if p1!=p2:
                head2head(individual1=p1, individual2=p2, nim_size=nim_size)
    
    best_contestor = sorted(contestors, key=lambda indv: indv['fitness'], reverse=True)[0]
    print('best contestor')
    print(best_contestor)
    return best_contestor
    

In [89]:
# take two parents and create one child
def crossover(parent1: dict, parent2: dict, mutation_prob: float) -> dict:
    rules_parent1 = [key for key in parent1.keys() if 'Rule' in key]
    
    child = {}

    for rule in rules_parent1:
        which_parent = random.randint(0,1)
        if which_parent == 0: # take rule from parent1
            child[rule] = parent1[rule]
        else:
            child[rule] = parent2[rule]
    
    child['fitness'] = 0
    rules = [key for key in child.keys() if 'Rule' in key]

    #mutation
    if random.random() < mutation_prob:
        rule = random.choice(rules)
        r1 = parent1[rule]
        r2 = parent2[rule]

        if rule == 'Rule1':
            mean = int((r1+r2)/2)
            child[rule] = mean

        else:
            mean_val_one = int((r1[0]+r2[0])/2)
            mean_val_two = int((r1[1]+r2[1])/2)
            child[rule] = [mean_val_one, mean_val_two]
    #logging.INFO('Child: {child}')  
    return child

     

In [90]:
indv1 = {'Rule1': 2, 'Rule2': [0,3], 'Rule3': [1,10], 'fitness':0}
indv2 = {'Rule1': 0, 'Rule2': [1,2], 'Rule3': [2,2], 'fitness':0}

crossover(indv1, indv2, 0.8)

{'Rule1': 2, 'Rule2': [0, 2], 'Rule3': [1, 10], 'fitness': 0}

# Evolution strategy described

1. Create inital population (with POPULATION_SIZE) where each player has the same set of rules but different parameters
2. Generate offspring where OFFSPRING_SIZE>>POPULATION_SIZE:
    - k individuals compete against each other and the fittest becomes a parent
    - Preform crossover between two parents and create one child
3. Play all children against each other to calculate fitness
4. Pick out the top l 
2. Evaluate:
    - Calculate fitness - all children plays agains each other (winner increase fitness)
3. Select the top k fittest children
4. Repeat 2-3 steps GENERATION amount of times


In [91]:
def create_offspring(population: list, nim_size: int, mutation_prob: float) -> list:
    offspring = []
    for i in range(OFFSPRING_SIZE):
        print('New offsring creation started')
        parent1 = tournament(population=population, nim_size=nim_size)
        parent2 = tournament(population=population, nim_size=nim_size)

        print("tournament")

        child = crossover(parent1, parent2, mutation_prob=mutation_prob)

        print("crossover")
        
        offspring.append(child)
    print('offspring')
    return offspring

In [92]:
def get_next_generation(offspring: list) -> list:
    for p1 in offspring:
        for p2 in offspring:
            calculate_fitness()
    
    best_k_offspring = sorted(offspring, key=lambda child: child['fitness'], reverse=True)[:POPULATION_SIZE]

    return best_k_offspring


In [100]:
# GAME
nim_size = 4
init_pop = init_population(nim_size=nim_size)

#for i in range(len(init_pop)):
#    for p in range(len(init_pop)):
#        head2head(individual1=init_pop[i], individual2=init_pop[p], nim_size=nim_size)

        

offspring = create_offspring(init_pop, nim_size, 0.2)



DEBUG:root:Coosing rule 3
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Coosing rule 3
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 1
DEBUG:root:Coosing rule 3
DEBUG:root:Coosing rule 3
DEBUG:root:Coosing rule 3
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Coosing rule 3
DEBUG:root:Coosing rule 3
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Coosing rule 3
DEBUG:root:Coosing rule 3
DEBUG:root:Choosing rule 2
DEBUG:root

New offsring creation started
H2H startes
H2H function
status: Initial board  -> <1 3 5 7>
status: After player 0 -> <1 3 5 3>
status: After player 1 -> <1 3 5 2>
status: After player 0 -> <1 3 4 2>
status: After player 1 -> <1 2 4 2>
status: After player 0 -> <1 2 3 2>
status: After player 1 -> <1 0 3 2>
status: After player 0 -> <0 0 3 2>
status: After player 1 -> <0 0 3 1>
status: After player 0 -> <0 0 2 1>
status: After player 1 -> <0 0 1 1>
status: After player 0 -> <0 0 0 1>
status: After player 1 -> <0 0 0 0>
winner ind 2
H2H function
status: Initial board  -> <1 3 5 7>
status: After player 0 -> <1 3 5 4>
status: After player 1 -> <1 1 5 4>
status: After player 0 -> <1 0 5 4>
status: After player 1 -> <1 0 1 4>
status: After player 0 -> <1 0 1 0>
status: After player 1 -> <0 0 1 0>
status: After player 0 -> <0 0 0 0>
winner ind 1
H2H function
status: Initial board  -> <1 3 5 7>
status: After player 0 -> <1 1 5 7>
status: After player 1 -> <1 1 3 7>
status: After player 0 -> <1 

DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Coosing rule 3
DEBUG:root:Coosing rule 3
DEBUG:root:Coosing rule 3
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Coosing rule 3
DEBUG:root:Coosing rule 3
DEBUG:root:Coosing rule 3
DEBUG:root:Coosing rule 3
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 1
DEBUG:ro

status: After player 0 -> <1 1 0 0>
status: After player 1 -> <0 1 0 0>
status: After player 0 -> <0 0 0 0>
winner ind 1
H2H function
status: Initial board  -> <1 3 5 7>
status: After player 0 -> <0 3 5 7>
status: After player 1 -> <0 3 5 1>
status: After player 0 -> <0 3 2 1>
status: After player 1 -> <0 1 2 1>
status: After player 0 -> <0 1 0 1>
status: After player 1 -> <0 0 0 1>
status: After player 0 -> <0 0 0 0>
winner ind 1
H2H function
status: Initial board  -> <1 3 5 7>
status: After player 0 -> <1 3 5 0>
status: After player 1 -> <1 2 5 0>
status: After player 0 -> <0 2 5 0>
status: After player 1 -> <0 2 4 0>
status: After player 0 -> <0 2 2 0>
status: After player 1 -> <0 1 2 0>
status: After player 0 -> <0 1 1 0>
status: After player 1 -> <0 0 1 0>
status: After player 0 -> <0 0 0 0>
winner ind 1
H2H function
status: Initial board  -> <1 3 5 7>
status: After player 0 -> <1 3 5 1>
status: After player 1 -> <1 0 5 1>
status: After player 0 -> <1 0 1 1>
status: After player 1

DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Coosing rule 3
DEBUG:root:Coosing rule 3
DEBUG:root:Coosing rule 3
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Coosing rule 3
DEBUG:root:Coosing rule 3
DEBUG:root:Coosing rule 3
DEBUG:r

...

status: After player 0 -> <0 1 3 0>
status: After player 1 -> <0 1 2 0>
status: After player 0 -> <0 1 1 0>
status: After player 1 -> <0 0 1 0>
status: After player 0 -> <0 0 0 0>
winner ind 1
H2H function
status: Initial board  -> <1 3 5 7>
status: After player 0 -> <1 3 1 7>
status: After player 1 -> <1 1 1 7>
status: After player 0 -> <0 1 1 7>
status: After player 1 -> <0 1 0 7>
status: After player 0 -> <0 1 0 6>
status: After player 1 -> <0 0 0 6>
status: After player 0 -> <0 0 0 5>
status: After player 1 -> <0 0 0 2>
status: After player 0 -> <0 0 0 1>
status: After player 1 -> <0 0 0 0>
winner ind 2
H2H function
status: Initial board  -> <1 3 5 7>
status: After player 0 -> <1 3 3 7>
status: After player 1 -> <1 0 3 7>
status: After player 0 -> <1 0 3 3>
status: After player 1 -> <1 0 2 3>
status: After player 0 -> <0 0 2 3>
status: After player 1 -> <0 0 2 1>
status: After player 0 -> <0 0 1 1>
status: After player 1 -> <0 0 0 1>
status: After player 0 -> <0 0 0 0>
winner ind 1

DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 1
DEBUG:root:Coosing rule 3
DEBUG:root:Coosing rule 3
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Coosing rule 3
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 1
DEBUG:root:Coosing rule 3
DEBUG:root:Coosing rule 3
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:

status: After player 1 -> <0 1 4 0>
status: After player 0 -> <0 1 3 0>
status: After player 1 -> <0 1 2 0>
status: After player 0 -> <0 1 1 0>
status: After player 1 -> <0 0 1 0>
status: After player 0 -> <0 0 0 0>
winner ind 1
H2H function
status: Initial board  -> <1 3 5 7>
status: After player 0 -> <1 0 5 7>
status: After player 1 -> <1 0 5 0>
status: After player 0 -> <1 0 4 0>
status: After player 1 -> <0 0 4 0>
status: After player 0 -> <0 0 3 0>
status: After player 1 -> <0 0 0 0>
winner ind 2
H2H function
status: Initial board  -> <1 3 5 7>
status: After player 0 -> <1 2 5 7>
status: After player 1 -> <1 2 5 3>
status: After player 0 -> <1 0 5 3>
status: After player 1 -> <0 0 5 3>
status: After player 0 -> <0 0 5 2>
status: After player 1 -> <0 0 5 1>
status: After player 0 -> <0 0 4 1>
status: After player 1 -> <0 0 3 1>
status: After player 0 -> <0 0 2 1>
status: After player 1 -> <0 0 1 1>
status: After player 0 -> <0 0 0 1>
status: After player 1 -> <0 0 0 0>
winner ind 2

DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Coosing rule 3
DEBUG:root:Coosing rule 3
DEBUG:root:Coosing rule 3
DEBUG:root:Coosing rule 3
DEBUG:root:Coosing rule 3
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:root:Choosing rule 1
DEBUG:root:Choosing rule 2
DEBUG:

status: After player 0 -> <0 0 0 0>
winner ind 1
H2H function
status: Initial board  -> <1 3 5 7>
status: After player 0 -> <1 3 3 7>
status: After player 1 -> <1 3 3 3>
status: After player 0 -> <1 3 1 3>
status: After player 1 -> <1 3 1 1>
status: After player 0 -> <1 1 1 1>
status: After player 1 -> <1 0 1 1>
status: After player 0 -> <1 0 1 0>
status: After player 1 -> <0 0 1 0>
status: After player 0 -> <0 0 0 0>
winner ind 1
H2H function
status: Initial board  -> <1 3 5 7>
status: After player 0 -> <1 3 5 0>
status: After player 1 -> <1 3 1 0>
status: After player 0 -> <1 2 1 0>
status: After player 1 -> <1 0 1 0>
status: After player 0 -> <0 0 1 0>
status: After player 1 -> <0 0 0 0>
winner ind 2
H2H function
status: Initial board  -> <1 3 5 7>
status: After player 0 -> <1 3 1 7>
status: After player 1 -> <1 3 1 1>
status: After player 0 -> <1 1 1 1>
status: After player 1 -> <1 0 1 1>
status: After player 0 -> <1 0 1 0>
status: After player 1 -> <0 0 1 0>
status: After player 0

In [94]:
def optimal_startegy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

Oversimplified match

In [95]:
logging.getLogger().setLevel(logging.DEBUG)

#strategy = (nim_sum_l({"p": 0.1}), optimal_startegy)

nim = Nim(11)
logging.debug(f"status: Initial board  -> {nim}")
player = 0

while nim:
    ply = nim_sum_l(nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")


AttributeError: 'dict' object has no attribute '_rows'